In [39]:
import numpy as np
import pandas as pd 
import re
from  collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

from sklearn import metrics

import emoji
import pickle

import joblib
from datetime import datetime
import json

In [17]:
df=pd.read_csv("raw_data.csv")

In [18]:
def paper_clean_str(text):
    text=re.sub("(@\S+)","mention",text)
    text=re.sub("(http\S+)","URL",text)
    text = text.replace("#","")
    text = text.replace("_"," ")
    return text


In [19]:
def my_clean_str(text):
    
    search = ['_','-','\n','\t','&quot;']

    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    text = text.replace('أ', 'ا')
    text = text.replace('آ', 'ا')
    text = text.replace('إ', 'ا')
    text = text.replace('أ', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], ' ')
        
    #removing mention,english letters , digits 
    text=re.sub("[^\u0621-\u064A\u0660-\u0669\s]","",text)
    
    #trim
    text = text.strip()

    return text

In [20]:
df["Text"]=df["Text"].apply(lambda Text:my_clean_str(Text))
for j in df.dialect.unique():
    all_country_words=[]
    exec(f"most_common_{j} = set()")
    for i in list((df[df.dialect == j]).Text):
        all_country_words.extend(i.split())
    for i in all_country_words:
        exec(f"most_common_{j}.add(i)")
all_words=[]
for j in df.dialect.unique():
    exec(f"l=list(most_common_{j})")
    all_words.extend(l)
most_repeted=Counter(all_words).most_common(22000)
stop_words=[]
for i in most_repeted:
        if i[1]>10:
            stop_words.append(i[0])
        else:
            break

In [21]:
x=df.Text
y=df.dialect
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.02, stratify=y, random_state=42)
vec3 = TfidfVectorizer(analyzer='char_wb', ngram_range=(1, 5),stop_words=stop_words)
clf3 = LinearSVC(random_state=0)
pipe = make_pipeline(vec3,clf3)
pipe.fit(x_train,y_train)


def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipe, x_test, y_test)

              precision    recall  f1-score   support

          AE       0.47      0.46      0.46       526
          BH       0.46      0.40      0.43       526
          DZ       0.63      0.57      0.60       324
          EG       0.74      0.88      0.80      1153
          IQ       0.66      0.67      0.66       310
          JO       0.46      0.39      0.42       558
          KW       0.52      0.57      0.55       842
          LB       0.68      0.75      0.72       552
          LY       0.69      0.73      0.71       730
          MA       0.78      0.63      0.70       231
          OM       0.57      0.44      0.50       382
          PL       0.53      0.59      0.56       875
          QA       0.53      0.57      0.54       621
          SA       0.43      0.48      0.45       537
          SD       0.74      0.66      0.70       289
          SY       0.51      0.38      0.44       325
          TN       0.78      0.51      0.61       185
          YE       0.55    

In [33]:
joblib.dump(pipe, 'model_pickle.joblib', compress=9)

['model_pickle.joblib']

In [40]:
y_pred = pipe.predict(x_test)
report = metrics.classification_report(y_test, y_pred)
status="Model Ready"
classes=pipe.classes_.tolist()
evaluation=report
model_status={}
model_status['status'] = status
model_status['timestamp'] = datetime.now().isoformat()
model_status['classes'] = classes
model_status['evaluation'] = evaluation
with open('model_status.json', 'w+') as file:
    json.dump(model_status, file, indent=2)

In [22]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipe, x_train, y_train,
                         scoring="accuracy", cv=10)

In [23]:
scores.mean()

0.5755924388715103

In [24]:
scores.std()

0.0022355429090720265

In [25]:
import eli5

In [26]:
eli5.show_weights(clf3, vec=vec3, top=20)

C:\Users\melsh\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [27]:
data_for_sampling=x_test.to_frame().join(y_test.to_frame())

In [28]:
for _, row in data_for_sampling.sample(5).iterrows():
    print(f"true label: {row['dialect']}")
    display(eli5.show_prediction(clf3, row['Text'], vec=vec3,))
    print("--"*50)

true label: BH


C:\Users\melsh\anaconda3\envs\nlp\lib\site-packages\eli5\formatters\text_helpers.py:29: RuntimeWarning: divide by zero encountered in double_scalars
  weight /= (end - start)


C:\Users\melsh\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: BH


C:\Users\melsh\anaconda3\envs\nlp\lib\site-packages\eli5\formatters\text_helpers.py:29: RuntimeWarning: divide by zero encountered in double_scalars
  weight /= (end - start)


...

C:\Users\melsh\anaconda3\envs\nlp\lib\site-packages\eli5\formatters\text_helpers.py:29: RuntimeWarning: divide by zero encountered in double_scalars
  weight /= (end - start)


C:\Users\melsh\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: TN


C:\Users\melsh\anaconda3\envs\nlp\lib\site-packages\eli5\formatters\text_helpers.py:29: RuntimeWarning: divide by zero encountered in double_scalars
  weight /= (end - start)


----------------------------------------------------------------------------------------------------
